# TFY4325/FY8904: Computational Physics (spring 2024)
## Assignment 2: The World of Quantum Mechanics

In this assignment we will study three different quantum mechanical systems, described by onedimensional Schrödinger equations. 

The first problem is the simple and well-known particle in a box, where we focus on scaling to dimensionless equations and comparing computational results to the exact
ones. 

The second problem describes a particle in a box with a potential barrier, where we do not have
many analytical results. 

In the third problem we learn how we can study a simple time-dependent quantum mechanical
platform. By considering a time-dependent Schr¨odinger equation for oscillating adjacent quantum wells
we find a time-dependent wavefunction that result in an integral equation. Finding the time-dependent
wavefunction, we predict the evolution of non-interacting particles that might exist in this system.

# 1. Particle in a box and dimensionless variables

The Schrödinger equation reads as follows

$$
i\hbar \frac{\partial}{\partial t}\Psi = \hat{H}\Psi
\tag{2.1}
$$
which is the baseline of quantum mechanics, describing how a system evolves in time given the Hamilationian operator $\hat{H}$ for the system.

Given some initial condition $\Psi_0(x)=\Psi(x,t = 0)$, and a time independent Hamiltonian, we get a general solution

$$
\Psi = exp(-\frac{it}{\hbar}\hat{H})\Psi_0 \tag{2.2}
$$

with the expontentional of the Hamiltionian being the series expansion. This equation describes the time evoolution of the system, and requires the eigenfunctions $\psi_n$ of the Hamiltionian for it to be useful. They can be extracted through the time-independent Schrödinger equation

$$
\hat{H}\psi_n = E_n \psi_n \tag{2.3}
$$

$E_n$ being the energy of the state $\psi_n$.

We want to study a single particle with mass $m$ confined in some 2D box with walls at $x=0$, and $x=L$.

The boundary conditions required for the wave function when the particle is confined in the box can be found by examining the potential that describes the box

$$
V(x) = 
\begin{cases} 
0 & \text{for } 0 < x < L \\
\infty & \text{otherwise}.
\end{cases} \tag{2.4}
$$

Given this potential, the boundary conditions are derived from the requirement that the wave function must be continuous and the probability density must be zero at the boundaries of the box. Mathematically, this is expressed as

$$
\Psi(x=0, t) = 0, \Psi(x=L,t)=0
$$

Further, we can construct the Hamiltonian

$$
\hat{H} = -\frac{\hbar^2}{2m} \frac{\partial^2}{\partial x^2} + V(x), \tag{2.5}
$$

which means that Eqs. (2.1) and (2.3), the time-dependent and time-independent Schrödinger equations, read

$$
-\frac{\hbar^2}{2m} \frac{\partial^2 \Psi}{\partial x^2} = i\hbar \frac{\partial}{\partial t} \Psi \tag{2.6}
$$

and inside the box, we have

$$
-\frac{\hbar^2}{2m} \frac{\partial^2 \psi_n}{\partial x^2} = E_n \psi_n \tag{2.7}
$$

To implement numerical algorithms, we must make the equations dimensionless. We define $t' = t/t_0$ and $x'=x/x_0$, with $t_0$ and $x_0$ being time and position measured in with the new coordinates $t'$ and $x'$. Using the chain rule for differentiation on $\Psi(x,t'(t))$, we get a relation between $\frac{\partial \Psi}{\partial t}$ and $\frac{\partial \Psi}{\partial t'}$.

Time derivative:
$$
\frac{\partial}{\partial t} = \frac{\partial t'}{\partial t} \frac{\partial}{\partial t'} = \frac{1}{t_0}\frac{\partial}{\partial t'}
$$

Positional derivative:
$$
\frac{\partial}{\partial x}= \frac{\partial x'}{\partial x} \frac{\partial}{\partial x'} = \frac{1}{x_0} \frac{\partial}{\partial x'} \Rightarrow \frac{\partial^2}{\partial x^2} = \frac{1}{x_0^2} \frac{\partial^2}{\partial x'^2}
$$

Plugging the results into Schrödinger equation $(2.6)$, we get

$$
i\hbar \frac{1}{t_0} \frac{\partial \Psi}{\partial t'} = -\frac{\hbar^2}{2m} \frac{\partial^2\Psi}{\partial x^2}
$$

Choosing $x_0=L$

$$
i \frac{\hbar}{t_0} \frac{\partial \Psi}{\partial t'} = -\frac{\hbar^2}{2mL^2}\frac{\partial^2 \Psi}{\partial x'^2}
$$

We compare both sides and get that $t_0 = 2mL^2/ \hbar$. We end up with the final dimensionless Schrödinger equation

$$
i \frac{\partial \Psi}{\partial t'} = -\frac{\partial^2 \Psi}{\partial x'^2} \tag{2.8}
$$

With our new variables $t'=\frac{\hbar}{2mL^2} t$ and $x'=x/L$, we can rewrite the eigenvalue problem in Eq. $(2.7)$ to

$$
-\frac{\partial^2 \psi_n}{\partial x'^2} = \lambda_n \psi_n \tag{2.9}
$$

We also need to consider the fact that we just transformed our variables, and that the boundary conditions are also changed. With $x' = x/L$, we then get

$$
\Psi(x'=0, t') = 0, \Psi(x'=1, t') = 0
$$